# Nutrition Image Analysis using CNN

### Importing Neccessary Libraries

In [4]:
import numpy as np#used for numerical analysis
import tensorflow #open source used for both ML and DL for computation
from tensorflow.keras.models import Sequential #it is a plain stack of layers
from tensorflow.keras import layers #A layer consists of a tensor-in tensor-out computation function
#Dense layer is the regular deeply connected neural network layer
from tensorflow.keras.layers import Dense,Flatten
#Faltten-used fot flattening the input or change the dimension
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Dropout #Convolutional layer
#MaxPooling2D-for downsampling the image
from keras.preprocessing.image import ImageDataGenerator


### Image Data Agumentation

In [ ]:
#setting parameter for Image Data agumentation to the training data
train_datagen = ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
#Image Data agumentation to the testing data
test_datagen=ImageDataGenerator(rescale=1./255)

### Loading our data and performing data agumentation

In [ ]:
#performing data agumentation to train data
x_train = train_datagen.flow_from_directory(
    r'C:\Users\Harithan\IBM_Proj\Dataset\TRAIN_SET',
    target_size=(64, 64),batch_size=5,color_mode='rgb',class_mode='sparse')
#performing data agumentation to test data
x_test = test_datagen.flow_from_directory(
    r'C:\Users\Harithan\IBM_Proj\Dataset\TEST_SET',
    target_size=(64, 64),batch_size=5,color_mode='rgb',class_mode='sparse') 

Found 730 images belonging to 4 classes.
Found 748 images belonging to 5 classes.


In [ ]:
print(x_train.class_indices)#checking the number of classes

{'APPLES': 0, 'BANANA': 1, 'ORANGE': 2, 'WATERMELON': 3}


In [ ]:
print(x_test.class_indices)#checking the number of classes

{'APPLES': 0, 'BANANA': 1, 'ORANGE': 2, 'PINEAPPLE': 3, 'WATERMELON': 4}


In [ ]:
from collections import Counter as c
c(x_train .labels)

Counter({0: 62, 1: 250, 2: 250, 3: 168})

### Creating the model

In [ ]:
# Initializing the CNN
classifier = Sequential()

# First convolution layer and pooling
classifier.add(Conv2D(32, (3, 3), input_shape=(64, 64, 3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))

# Second convolution layer and pooling
classifier.add(Conv2D(32, (3, 3), activation='relu'))

# input_shape is going to be the pooled feature maps from the previous convolution layer
classifier.add(MaxPooling2D(pool_size=(2, 2)))

# Flattening the layers
classifier.add(Flatten())

# Adding a fully connected layer
classifier.add(Dense(units=128, activation='relu'))
classifier.add(Dense(units=5, activation='softmax')) # softmax for more than 2



In [ ]:
classifier.summary()#summary of our model

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 6272)              0         
                                                                 
 dense (Dense)               (None, 128)               8

### Compiling the model

In [ ]:
# Compiling the CNN
# categorical_crossentropy for more than 2
classifier.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy']) 

## Fitting the model

In [ ]:
classifier.fit_generator(
        generator=x_train,steps_per_epoch = len(x_train),
        epochs=20, validation_data=x_test,validation_steps = len(x_test))# No of images in test set

C:\Users\Harithan\AppData\Local\Temp\ipykernel_2984\549542485.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  classifier.fit_generator(


Epoch 1/20
146/146 [==============================] - 34s 205ms/step - loss: 0.8986 - accuracy: 0.6178 - val_loss: 0.9518 - val_accuracy: 0.6056
Epoch 2/20
146/146 [==============================] - 16s 108ms/step - loss: 0.5531 - accuracy: 0.7658 - val_loss: 0.5908 - val_accuracy: 0.7701
Epoch 3/20
146/146 [==============================] - 16s 109ms/step - loss: 0.4971 - accuracy: 0.7973 - val_loss: 0.7736 - val_accuracy: 0.6912
Epoch 4/20
146/146 [==============================] - 16s 110ms/step - loss: 0.4943 - accuracy: 0.8110 - val_loss: 0.9634 - val_accuracy: 0.6578
Epoch 5/20
146/146 [==============================] - 14s 98ms/step - loss: 0.4215 - accuracy: 0.8274 - val_loss: 0.8180 - val_accuracy: 0.7353
Epoch 6/20
146/146 [==============================] - 16s 111ms/step - loss: 0.3976 - accuracy: 0.8493 - val_loss: 0.7120 - val_accuracy: 0.7366
Epoch 7/20
146/146 [==============================] - 16s 108ms/step - loss: 0.3702 - accuracy: 0.8658 - val_loss: 1.0481 - val_acc

### Saving our model

In [ ]:
# Save the model
classifier.save('nutrition.h5')

### Predicting our results

In [ ]:
from tensorflow.keras.models import load_model
import keras
from tensorflow.keras.preprocessing import image
model = load_model("nutrition.h5") #loading the model for testing

In [ ]:
img = keras.utils.load_img(r'C:\Users\Harithan\IBM_Proj\flask\uploads\Test_Image5.jpg',
                     grayscale=False,target_size= (64,64))#loading of the image
x = image.img_to_array(img)#image to array
x = np.expand_dims(x,axis = 0)#changing the shape
pred = (model.predict(x) > 0.5)#predicting the classes
pred

1/1 [==============================] - 0s 346ms/step


array([[False, False,  True, False, False]])

In [ ]:
index=['APPLES', 'BANANA', 'ORANGE','PINEAPPLE','WATERMELON']
result=str(index[pred[0]])
result

TypeError: only integer scalar arrays can be converted to a scalar index